In [44]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-mar-2021/train.csv
/kaggle/input/tabular-playground-series-mar-2021/test.csv


In [45]:
train = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,target
0,0,A,I,A,B,B,BI,A,S,Q,...,0.759439,0.795549,0.681917,0.621672,0.592184,0.791921,0.815254,0.965006,0.665915,0
1,1,A,I,A,A,E,BI,K,W,AD,...,0.386385,0.541366,0.388982,0.357778,0.600044,0.408701,0.399353,0.927406,0.493729,0
2,2,A,K,A,A,E,BI,A,E,BM,...,0.343255,0.616352,0.793687,0.552877,0.352113,0.388835,0.412303,0.292696,0.549452,0
3,3,A,K,A,C,E,BI,A,Y,AD,...,0.831147,0.807807,0.800032,0.619147,0.221789,0.897617,0.633669,0.760318,0.934242,0
4,4,A,I,G,B,E,BI,C,G,Q,...,0.338818,0.277308,0.610578,0.128291,0.578764,0.279167,0.351103,0.357084,0.328960,1


In [46]:
#### Number of unique values in each categorical column

unique_catcol_list = []

for i in range(19):
    unique_val = train['cat'+str(i)].nunique()
    unique_catcol_list.append('cat'+str(i)+':'+str(unique_val))
print(unique_catcol_list)

['cat0:2', 'cat1:15', 'cat2:19', 'cat3:13', 'cat4:20', 'cat5:84', 'cat6:16', 'cat7:51', 'cat8:61', 'cat9:19', 'cat10:299', 'cat11:2', 'cat12:2', 'cat13:2', 'cat14:2', 'cat15:4', 'cat16:4', 'cat17:4', 'cat18:4']


In [47]:
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
test.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10
0,5,A,F,A,A,F,BI,A,AH,AX,...,0.735690,0.578366,0.723154,0.228037,0.356227,0.551249,0.655693,0.598331,0.359987,0.947489
1,6,A,H,C,A,E,AB,I,F,N,...,0.313703,0.928885,0.516602,0.600169,0.795224,0.248987,0.654614,0.347944,0.565520,0.388580
2,8,A,N,C,A,F,AB,A,AH,BC,...,0.448201,0.424876,0.344729,0.242073,0.270632,0.746740,0.335590,0.341238,0.252289,0.411592
3,9,B,L,C,A,F,BI,A,E,AX,...,0.666092,0.598943,0.561971,0.806347,0.735983,0.538724,0.381566,0.481660,0.348514,0.325723
4,11,A,F,A,B,F,BI,A,AH,I,...,0.772229,0.479572,0.767745,0.252454,0.354810,0.178920,0.763479,0.562491,0.466261,0.585781


In [48]:
test.columns == train.drop('target',axis = 'columns').columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True])

In [49]:
print(test.isna().sum().sum())  ### No missing values in train data
print(test.isna().sum().sum())  ### No missing values in test data

0
0


In [50]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import make_column_transformer
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline


In [19]:
### Linear Model

X = train.drop(['target','id'],axis = 'columns')
y = train.target
test = test.drop('id',axis = 'columns')

##Extracting categorical columns and numerical col list for preprocessing
cat_col_list = X.select_dtypes(include = ['object']).columns.to_list()
num_col_list = X.select_dtypes(exclude = ['object']).columns.to_list()

preprocessor = make_column_transformer((StandardScaler(),num_col_list),
                                   (OneHotEncoder(handle_unknown = 'ignore'),cat_col_list))

model = LinearRegression()
pipeline = make_pipeline(preprocessor,model)
pipeline.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cat0', 'cat1', 'cat2',
                                                   'cat3', 'cat4', 'cat5',
                                                   'cat6', 'cat7', 'cat8',
                                                   'cat9', 'cat10', 'cat11',
                                                   'cat12', '

In [20]:
ypred = pipeline.predict(test)
test = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
df = pd.DataFrame()
df['id'] = test['id']
df['target'] = ypred
df.to_csv('sub.csv',index = False)

In [21]:
##XGBoost model
import xgboost
from xgboost import XGBRegressor
model = XGBRegressor()
pipeline = make_pipeline(preprocessor,model)
pipeline.fit(X,y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('standardscaler',
                                                  StandardScaler(),
                                                  ['cont0', 'cont1', 'cont2',
                                                   'cont3', 'cont4', 'cont5',
                                                   'cont6', 'cont7', 'cont8',
                                                   'cont9', 'cont10']),
                                                 ('onehotencoder',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['cat0', 'cat1', 'cat2',
                                                   'cat3', 'cat4', 'cat5',
                                                   'cat6', 'cat7', 'cat8',
                                                   'cat9', 'cat10', 'cat11',
                                                   'cat12', '

In [22]:
ypred1 = pipeline.predict(test.drop('id',axis = 'columns'))
test1 = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
df = pd.DataFrame()
df['id'] = test1['id']
df['target'] = ypred1
df.to_csv('sub_XGBR.csv',index = False)

In [51]:
X = train.drop(['target','id'],axis = 'columns')
y = train.target
test = test.drop('id',axis = 'columns')

##Extracting categorical columns and numerical col list for preprocessing
cat_col_list = X.select_dtypes(include = ['object']).columns.to_list()
num_col_list = X.select_dtypes(exclude = ['object']).columns.to_list()

preprocessor = make_column_transformer((StandardScaler(),num_col_list),
                                   (OneHotEncoder(sparse = False,handle_unknown = 'ignore'),cat_col_list))

In [52]:
import numpy as np

In [53]:
X = preprocessor.fit_transform(X)
test = preprocessor.transform(test)

In [38]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization


In [56]:
## NN using tensor flow

model3 = Sequential([Dense(1024,activation = 'relu',input_shape = [X.shape[1]]),
                     BatchNormalization(),
                     Dropout(0.3),
                     Dense(512, activation = 'relu'),
                     BatchNormalization(),
                     Dropout(0.3),
                     Dense(256,activation = 'relu'),
                     BatchNormalization(),
                     Dropout(0.3),
                     Dense(1)
])
model3.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 1024)              650240    
_________________________________________________________________
batch_normalization (BatchNo (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_17 (Dense)             (None, 512)               524800    
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 256)              

In [57]:
model3.compile(loss = 'mae',optimizer = 'adam')
model3.fit(X,y,batch_size = 256,epochs = 10)

Epoch 1/10
1172/1172 [==============================] - 40s 33ms/step - loss: 0.4089
Epoch 2/10
1172/1172 [==============================] - 39s 33ms/step - loss: 0.1836
Epoch 3/10
1172/1172 [==============================] - 40s 34ms/step - loss: 0.1759
Epoch 4/10
1172/1172 [==============================] - 41s 35ms/step - loss: 0.1708
Epoch 5/10
1172/1172 [==============================] - 42s 36ms/step - loss: 0.1687
Epoch 6/10
1172/1172 [==============================] - 40s 34ms/step - loss: 0.1657
Epoch 7/10
1172/1172 [==============================] - 39s 34ms/step - loss: 0.1653
Epoch 8/10
1172/1172 [==============================] - 40s 34ms/step - loss: 0.1631
Epoch 9/10
1172/1172 [==============================] - 39s 33ms/step - loss: 0.1633
Epoch 10/10
1172/1172 [==============================] - 39s 33ms/step - loss: 0.1606


In [60]:
ypred3 = model3.predict(test)

In [62]:
test2 = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')
df = pd.DataFrame()
df['id'] = test2['id']
df['target'] = ypred3
df.to_csv('sub_TF.csv',index = False)

In [ ]:
## NN using pytorch
X = train.drop(['target','id'],axis = 'columns')
y = train.target
test = test.drop('id',axis = 'columns')

##Extracting categorical columns and numerical col list for preprocessing
cat_col_list = X.select_dtypes(include = ['object']).columns.to_list()
num_col_list = X.select_dtypes(exclude = ['object']).columns.to_list()

preprocessor = make_column_transformer((StandardScaler(),num_col_list),
                                       (OneHotEncoder(sparse = False,handle_unknown = 'ignore'),cat_col_list))

X = preprocessor.fit_transform(X)
test = preprocessor.transform(test)

In [69]:
from torch import nn
from torch.nn import Linear
from torch.nn import Dropout,BatchNorm1d

In [ ]:
model4 = nn.Sequential([nn.Linear(X.shape[1],500),
                       nn.BatchNorm1d(500),
                        nn.Dropout(0.3),
                        nn.Linear(500,300),
                        nn.BatchNorm1d(300),
                        nn.Dropout(0.3),
                        nn.Linear(100,1)
                       
                       ])